In [1]:
import math
import numpy as np
import re
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
#Display Requirements

height = 8.4 #inches
width = 11.2 #inches
resolution_height = 480 #pixels
resolution_width = 640 #pixels
depth = 18 #inches
degree_va = 3* (math.pi)/180 #3 Degrees of visual angle
deviation_inches = depth * math.tan(degree_va)

DPI = (resolution_height**2 + resolution_width**2)**(0.5)/((height**2 + width**2))**(0.5)

deviation = deviation_inches * DPI

In [ ]:
#loading the data
PATH = "/content/drive/MyDrive/QE_Project/data_local_part_1_15_6_2021_16_49_27_1to1_event.xlsx"

data = pd.read_excel(PATH)
df = pd.DataFrame(data)
data_array = df.to_numpy()
data_splt_1 = data_array[:-1, 0:3]
data_splt_2 = data_array[1:, 3:]
data_merge = np.hstack((data_splt_1, data_splt_2))
data = data_merge[::2]
#Discard first frame because the values for the eye gaze is not correct. (gaze_cnt = 0)
data = data[1:, :]
QE_period = 100 #ms
data[:, 2] = (data[:, 91] - data[0, 91]) * 1000

#Convert the x and y positions of the LPOG to pixels
data[:, 105] = (data[:, 105]/100)*resolution_width
data[:, 106] = (data[:, 106]/100)*resolution_height

data_filtered = np.hstack((data[:, 1:3], data[:, 47:54], data[:, 56:63], data[:, 65:69], data[:, 105:108]))
nan_indexes = [i for i in range(len(data_filtered)) if np.isnan(data_filtered[i, 20])]
data_filtered = np.delete(data_filtered, nan_indexes, 0)

cX = data_filtered[0, 20]
cY = data_filtered[0, 21]

i = 0
fixations = []
while (i < len(data_filtered) - 1):

  cX, cY = data_filtered[i, 20], data_filtered[i, 21]
  fixation = return_first_fixation(cX, cY, i, deviation, data_filtered)
  print(fixation)
  if(fixation[-1] > QE_period):
    fixations.append(fixation)
  
  i = fixation[0]

# print(fixations)

In [13]:
#takes in center coordinates, starting index, deviation and returns the first (likely) fixation 

def return_first_fixation(cX, cY, i, deviation, data):
  
  j = i + 1
  fixation_time = data[i, 1]
  frame_number = data[i, 0]

  for j in range(i + 1, len(data)):

    jX, jY = data[j, 20], data[j, 21]
    dX, dY = abs(jX - cX), abs(jY - cY)
    distance = (dX**2 + dY**2)**(0.5)

    if (distance < deviation):
      fixation_time = data[j, 1]
      frame_number = data[j, 0]

    else:
      duration = fixation_time - data[i, 1]
      array = [j, data[i, 0], data[j - 1, 0], duration]
      return array

  duration = fixation_time - data [j - 1, 1]
  array = [j, data[i, 0], data[j - 1, 0], duration]
  return array

# takes in the dataset, deviation and QE_period and returns a list of all fixations (likely) in the dataset.

def return_possible_fixations(data, deviation, QE_period):
  i = 0
  fixations = []
  while (i < len(data_filtered) - 1):

    cX, cY = data_filtered[i, 20], data_filtered[i, 21]
    fixation = return_first_fixation(cX, cY, i, deviation, data_filtered)
    print(fixation)
    if(fixation[-1] > QE_period):
      fixations.append(fixation)
    
    i = fixation[0]

  return fixations